<a href="https://colab.research.google.com/github/JamesLabUofT/GEE_Workshop/blob/main/scripts/working_with_era5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with ERA5 Climate Data in Google Earth Engine Using Python

### Module Overview:

This module introduces participants to the fundamentals of accessing, processing, and extracting climate data—specifically ERA5 reanalysis data—using Google Earth Engine (GEE) and Python. Through hands-on coding exercises in Google Colab, participants will learn how to build scalable workflows for climate data analysis relevant to ecological and environmental research.

### Learning Objectives:

By the end of this module, participants will be able to:

* Understand the structure and content of ERA5 datasets available in GEE.
* Use Python and the Earth Engine API to access and filter ERA5 data by time and region.
* Extract and visualize climate variables (e.g., precipitation, temperature) over custom geographic areas.
* Export processed data for further analysis or reporting.

### Topics Covered:

**Introduction to ERA5 Data**

1. What is ERA5? - Key variables and temporal resolution
2. Accessing ERA5 in Google Earth Engine


Using Google Colab for cloud-based analysis
Installing and importing required packages (earthengine-api, geemap, pandas, matplotlib)

**Data Acquisition**

1. Filtering ERA5 ImageCollections by date, region or metadata.
2. Selecting relevant climate variables (e.g., total_precipitation, temperature_2m)

**Data Processing**

1. Reducing data over regions of interest (e.g., national parks, watersheds)
2. Aggregating data temporally (monthly, yearly averages)
3. Converting units and formatting results

**Data Extraction and Visualization**

1. Exporting time series data to Pandas DataFrames**
2. Creating plots and maps using matplotlib and geemap
3. Exporting data to CSV or Google Drive

**Custom Functions to Extract Data**

1. Function to extract parameters for FWI calculations


## Introduction to ERA5

1. What is ERA5?

ERA5 is the fifth version of global climate data reanalysis created by the Copernicus Climate Change Service at the European Centre for Medium-Range Weather Forecasts (ECMWF). It offers hourly data on various climate-related factors in the atmosphere, land, and oceans, starting from 1940 up to today.

**Native resolution of 9km, regridded to 0.1 x 0.1 degrees (approximately 11km) in GEE for ERA5 Land.**

Temporal Aggregations:
  1. Hourly - Houly data for key variab;es
  2. Daily - Daily aggregates of the core variables,
  3. Monthly - Monthly averages and extremes of key variables


Key Variables

*   Temperature (measured at 2m)
*   Precipitation
*   Wind speed
*   Dewpoint Temperature
*   Pressure
*   Radiation Fluxes
*   Soil moisture
*   Snow Cover




2. Accessing ERA5 via python API

In order to start working with the python GEE API, we need to initilaze our environment. We need to do this at the beginning of every new script. As a reminder, we do so with ee.Authenticate() and/or ee.Initialize()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project="ee-jandrewgoldman") #ee-userid

For this workshop, we will also be working with geopandas, matplotlib, geemap, pandas, contextily

In [ ]:
#!pip install contextily
#!pip install geemap
#!pip install ipyleaflet



In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
import geemap
import ipyleaflet




For example, lets read in era5_land_hourly

In [ ]:
# read in era5_land_hourly
era5_land_hourly = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')

#read in era5 land_daily
era5_land_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')

# read in era5 land_monthly
era5_land_monthly = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')


# Data Acquisition

Now that we have the data loaded, lets see what we are working with.

Lets try to look at the image collections properties (i.e., columns).

---



In [ ]:
# Get the information about the collection, including its properties
collection_info = era5_land_hourly.getInfo()

# Print the collection properties
print('Collection Properties:')
for prop, value in collection_info.get('properties', {}).items():
    print(f'{prop}: {value}')


As you can see above, our query returns an error message. This is because we are trying to get information on a large number of images/properties.

In [ ]:
# For example, the number of images in the collection:
count = era5_land_hourly.size().getInfo()
print(f'\nNumber of images in collection: {count}')

When reading in data from image or feature collections it is always a good idea to filter the collection. Remember, image/feature collections are very large. Since we are working we a cloud-based system (and for free), we are unable to print or visualize very large objects. It is easier to handle filtered data.

In [ ]:

# Get the date range
date_range = era5_land_hourly.reduceColumns(ee.Reducer.minMax(), ['system:time_start'])

# Extract min and max dates
min_date_ms = date_range.get('min').getInfo()
max_date_ms = date_range.get('max').getInfo()

# Convert milliseconds to ee.Date objects and then to human-readable format
min_date = ee.Date(min_date_ms).format('YYYY-MM-dd').getInfo()
max_date = ee.Date(max_date_ms).format('YYYY-MM-dd').getInfo()

print(f"Image Collection Date Range: From {min_date} to {max_date}")



When filtering the data prior to running any queries there are three types of filtering methods.

1. Filter by metadata: You can apply a filter on the image metadata using filters such as ee.Filter.eq(), ee.Filter.lt() etc. You can filter by PATH/ROW values, Orbit number, Cloud cover etc.
2. Filter by date: You can select images in a particular date range using filters such as ee.Filter.date().
3. Filter by location: You can select the subset of images with a bounding box, location or geometry using the ee.Filter.bounds(). You can also use the drawing tools to draw a geometry for filtering.

Let's first load in the AOI.

For this excerise we will be using Woodland Caribou Provincial Park. The geojson can be found in the github and can be loaded directly using the url.

In [ ]:
woodland_fpath = "https://raw.githubusercontent.com/JamesLabUofT/GEE_Workshop/main/docs/shapefiles/woodland_caribou_provincial_park.shp"

aoi = gpd.read_file(woodland_fpath)
aoi

It is always good practice to check the crs and change to wgs 84

Lets visualize the AOI.


In [ ]:
# Plot the AOI using geopandas built-in plotting function
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
aoi.plot(ax=ax)
# Add a basemap for context
ctx.add_basemap(ax, crs=aoi.crs.to_epsg())
plt.title("Area of Interest: Woodland Caribou Provincial Park")
plt.show()

Since we read in the geojson (or feature using GEE terminology) locally as a geodataframe (gdf) using geopandas, we have to transform the gdf into a earth engine object in order to work with it on the server.

We can use geemap for this.

Geemap has numerous functions that allow use to convert various data types to ee objects.
shp_to_ee() = shapefiles
geojson_to_ee() = geojson
gdf_to_ee() = GeoDataFrame
csv_to_ee() = csv's

We will use gdf_to_ee

In [ ]:
aoi_ee = geemap.gdf_to_ee(aoi) # First select's the first feature in the feature collection, since we only have 1 feature, we want it to be a feature and not a feature collection
aoi_ee


In [ ]:
# Verify the type of aoi_ee
print(f"Type of aoi_ee: {type(aoi_ee)}")

# Create a new map centered and zoomed to the AOI
# We can use the centroid of the AOI's geometry for centering
centroid = aoi_ee.geometry().centroid(1).getInfo()['coordinates']
# geemap often automatically zooms to the layer extent when added,
# but explicitly setting a zoom level can help if needed.
# A zoom level between 8 and 10 is usually good for a provincial park.
zoom_level = 9

m = geemap.Map(center=(centroid[1], centroid[0]), zoom=zoom_level)

# Visualization parameters are not strictly needed for feature collections/geometries,
# but we can add a simple outline color if the default is hard to see.
aoi_vis = {'color': 'FF0000'} # Red outline

# Add the AOI layer using aoi_ee
m.add_layer(aoi_ee, aoi_vis, "Woodland Caribou Provincial Park")

# Add the layer manager to easily toggle layers
m.add_layer_manager()

m

Using the woodland caribou AOI we can clip the era5 land image collection to the AOI and specify date ranges.

We always filter by the AOI first. This significantly reduces the size of the image collection you're working with, making subsequent filtering steps more efficient.

Once you have images relevant to your AOI, you can then filter them by date or date range to select the specific images you need for your analysis.


In [ ]:
# Filter by AOI and date range
era5_land_daily_filtered = era5_land_daily.filterDate('2023-07-01', '2023-07-31').filterBounds(aoi_ee)

print(f"Filtered collection size: {era5_land_daily_filtered.size().getInfo()}")




Let's inspect the image collection





In [ ]:
# Get the information about the collection
era5_land_daily_filtered

The image collection is contains 30 images corresponding to each day, with 150 bands (variables) per image.

Images are split into bands and properties, bands contains the variables of interest and properties contain the metadata for the image, in this case the day, month and year. The system:time_end and system:time_start is the system time stanp im miliseconds since the UNIX epoch.

If we want to look at one date in particular, we can filter the image collection by metadata.


In [ ]:
era5_july_10 = era5_land_daily_filtered.filter(ee.Filter.eq('day', 10))
era5_july_10

If we want to look at the temperature for that day, we can select the temperature band

In [ ]:
era5_july_10_temp = era5_july_10.select('temperature_2m')
era5_july_10_temp

Lets say we want to get the average temperature over the time range for Woodland Caribou, to do this, we can use reducers.

Reducers allow us to reduce an Image Collection to a single image based on a variable of interest. There are different ways we can reduce an Image Collection.
*   Mean
*   Median
*   Mode
*   Sum
*   MinMax
*   Quartiles

There are two ways we can apply a reducer over the image

The first options to reduce an image collection, we can call .median()
This option reduces the image collection to an image.

At each location in the output image, in each band, the pixel value is the median of all unmasked pixels in the input imagery (the images in the collection). In the previous example, median() is a convenience method for the following cal

In [ ]:
median_temp_img =  era5_land_daily_filtered.select("temperature_2m").median()
median_temp_img

After filtering an image, **it is always a good idea to clip the image.** This is because although we filtered the dataset in the previous step, an Image Collection is a mosaic of tiles (in this case 11km tiles), therefore it will retain all tiles that cross the bounds of our image (i.e., it does not mask areas outside the boundary). Clipping on the other hand, can only be done for an image, and crops the image to the boundary of our feature (AOI) and masks all values that fall outside of it.

In [ ]:
clipped_temp_img = median_temp_img.clip(aoi_ee)


In era5 temperature is in kelvin not celcius. We can easily change this by subtracting 273.15 from the image.

In this step we will also rename the band from "temperatue_2m" to "temperature_celcius"

In [ ]:
clipped_temp_img_celcius = clipped_temp_img.subtract(273.15).rename('temperature_celsius')

We can visualize the image

In [ ]:
vis_params = {
  'min': 16,
  'max': 18,
  'palette': [ '#ffff00', '#ffcc00','#ff9900','#ff6600','#ff3300','#ff0000']
}

m.addLayer(clipped_temp_img_celcius, vis_params, 'Temperature')
m

Now say we want to get the min and max temperature over throughout the image, we can do this using the other type of reducer, .reduceRegions()

This reduce calculates numerous metrics over the area of interest and returns the value as a dictionary object.

This is different that the previous function in that we do not need to clip the image after. Instead we supply the reducer a region of interest, and it reduces all pixels in the image over the given region.

In [ ]:
median_temp_img_celcius = median_temp_img.subtract(273.15).rename('temperature_celsius')

woodland_reduced = median_temp_img_celcius.reduceRegion(
            reducer=ee.Reducer.minMax(), # could be type .mean(), .mode()...
            geometry=aoi_ee.geometry(), # specify the geometry
            scale=1000, # set scale usually depends on the grain of the image
            maxPixels=1e9) # setting maxPixels to smallest number allows..

woodland_reduced

In Google Earth Engine (GEE), a dictionary is a data structure that stores key-value pairs. It’s similar to dictionaries in other programming languages like Python or JavaScript. In GEE, dictionaries are represented using the ee.Dictionary class.

Key Concepts:
Keys are strings that identify each entry.
Values can be numbers, strings, lists, other dictionaries, or Earth Engine objects like ee.Number, ee.String, etc.

Instead of calculating the median value across our time period, we can also get more fine grained information by computing the trends over time.

To do so, we will need to add a time band to the image and then fit a lienar regression.

To add the time band we will create a function a .map() over each image in our image collection

In [ ]:
# function for time band
def add_date_band(image):
  # Get the day of year
  doy = image.date().getRelative('day', 'year')
  # Create a constant image with the DOY value
  doy_band = ee.Image.constant(doy).uint16().rename('doy')
  # Mask using any valid band (e.g., 'temperature_2m_max')
  doy_band = doy_band.updateMask(image.select('temperature_2m').mask())
  # Add the DOY band to the image
  return image.addBands(doy_band)

collection_with_time = era5_land_daily_filtered.map(add_date_band)
collection_with_time

For this we will use the original filtered image collection, we also need to convert the temperature to celcius. We will write a function for this as well.

In [ ]:
# function for kelvin_to_celcius
def kelvin_to_celcius(image):
    celsius = image.select('temperature_2m').subtract(273.15).rename('temperature_celsius')
    return image.addBands(celsius)

# apply function
celsius_collection = collection_with_time.map(kelvin_to_celcius)
celsius_collection

Run the regression

In [ ]:
trend = celsius_collection.select(['doy', 'temperature_celsius']).reduce(ee.Reducer.linearFit())


Visualize the trend - clip image collection by region of interest

In [ ]:
trend_image = trend.select('scale').clip(aoi_ee)

m.addLayer(trend_image.select('scale'), {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}, 'Temperature Trend (°C/day)'
)
m

## Data extraction and visualization

Alhough we govered visualization with geemap while working through data acquisition, we will now go through extracting data in different formats (i.e., images, csvs)

In my opinion, this is the most important aspect of GEE. Organizing data properly can be an honerous task, especially because while we can compute large items rather efficiently on the cloud, extracting these data from the cloud can be laborious. In some cases, our queries may be too large and fail. It is therefore important to understand some tips and tricks to extract data.

For this exercise we will be working with feature collections. If you remember, a feature collection is a group of geographic features (like points, lines, or polygons), each with associated attribute data. It’s used to represent vector data in Earth Engine.

Why use feature collections?
They allow you to organize and analyze spatial data by region or category—ideal for tasks like zonal statistics, sampling, or filtering by attributes.

How are they structured?
Each feature in the collection has a geometry (location or shape) and properties (like name, area, or type). The collection acts like a table of spatial records.

Where Do They Come From?
You can load them from Earth Engine’s public datasets, import your own shapefiles or GeoJSON, or create them manually in code.

In this section we will:
*  use feature collection to calculate zonal statistics
*  Filtering and grouping attributes
*  map functions over multiple geometries
*  Exporting results (e.g., CSV or shapefile)

We will use a feature collection of 3 large provincial parks - Woodland Caribou, Quetico and Algonquin

In [ ]:
fc_fpath = "https://raw.githubusercontent.com/JamesLabUofT/GEE_Workshop/main/docs/shapefiles/large_provincial_parks.shp"

lrg_parks = gpd.read_file(fc_fpath)
lrg_parks

Convert to ee odject

In [ ]:
lrg_parks_ee = geemap.gdf_to_ee(lrg_parks)
lrg_parks_ee

In this excerise we will look at the maximum daily temperature and total precipitation for summer of 2000 and 2023 and calculate the mean over time



In [ ]:
era5_land_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')

Set date ranges

In [ ]:
start_2000 = '2000-06-01'
end_2000 =  '2000-08-31'

start_2023= '2023-06-01'
end_2023= '2023-08-31'


Select the variables of interest





In [ ]:
era5_land_daily = era5_land_daily.select(['temperature_2m_max', 'total_precipitation_sum'])


Filter by date range

In [ ]:
range_2000 = era5_land_daily.filterDate(start_2000, end_2000)
range_2023 = era5_land_daily.filterDate(start_2023, end_2023)




Lets investigate differences in mean and total precipitation over these date ranges

We first extract the specific bands for maximum temperature and total precipitation from each filtered collection.

Selecting only the needed bands simplifies the dataset and focuses the analysis on relevant variables.

In [ ]:
# Select temperature and precipitation bands
temp_2000 = range_2000.select('temperature_2m_max')
precip_2000 = range_2000.select('total_precipitation_sum')

temp_2023 = range_2023.select('temperature_2m_max')
precip_2023 = range_2023.select('total_precipitation_sum')




Description: These lines compute the mean temperature and total precipitation for each time period.
Reasoning: Reducing the data to summary statistics allows for direct comparison between the two periods.

In [ ]:
# Reduce to summary images
mean_temp_2000 = temp_2000.mean().rename('mean_temperature_2000')
total_precip_2000 = precip_2000.sum().rename('total_precipitation_2000')

mean_temp_2023 = temp_2023.mean().rename('mean_temperature_2023')
total_precip_2023 = precip_2023.sum().rename('total_precipitation_2023')



Description: This merges all four summary bands into a single image for easier analysis and export.
Reasoning: Combining results into one image simplifies downstream processing and visualization.

In [ ]:
# Combine into one image for comparison
comparison_image = mean_temp_2000 \
  .addBands(total_precip_2000) \
  .addBands(mean_temp_2023) \
  .addBands(total_precip_2023)



Lets look at the metadata

In [ ]:
# Print image metadata
comparison_image

Now lets extract the mean temperature and precipitation for each image in the collection for each park. We will return the value as a column in a dataframe

In [ ]:

zonal_stats = comparison_image.reduceRegions(
    collection=lrg_parks_ee,
    reducer=ee.Reducer.mean(),
    scale=1000
)
zonal_stats

In [ ]:
# Apply the processing function to each feature in the collection
processed_collection = zonal_stats.flatten()

#
processed_collection


In [ ]:

    # Convert the collection to a list of dictionaries
    data_list = processed_collection.getInfo()['features']

    # Extract properties from each feature
    data_dicts = [feature['properties'] for feature in data_list]

    # Create a DataFrame
    df = pd.DataFrame(data_dicts)

    df

merge the two filtered collections

In [ ]:
combined = range_2000.merge(range_2023)


Sort by time to maintain a chronological order

In [ ]:
combined = combined.sort('system:time_start')



Lets check to see how many images we have

In [ ]:
# Print the number of images in the combined collection
print('Total images:', combined.size().getInfo())

Lets calculate the mean for each

## Custom Function To Extract FWI Data

In [ ]:
def extract_era5_land_data(feature_collection, year):
    """This function takes a feature collection and a year as YYYY and returns:
       a pandas dataframe with a row for date, t, rh,ws, ppt.
       start date and end date are set in the function to April 30 - August 31 of each year.
       This can be adjusted in the function under #Define the date range

    """
    # Define the date range
    start_date = f'{year}-04-30'
    end_date = f'{year}-08-31'

    # Load the ERA5-Land hourly dataset
    era5_land = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')

    # Filter the dataset by date range and time (13:00)
    era5_filtered = era5_land.filterDate(start_date, end_date).filter(ee.Filter.eq('hour', 13))

    # Define the bands to extract
    bands = ['temperature_2m', 'total_precipitation', 'u_component_of_wind_10m',
             'v_component_of_wind_10m', 'dewpoint_temperature_2m']

    # Select the bands
    era5_selected = era5_filtered.select(bands)

    # Calculate relative humidity and wind speed
    def calculate_additional_bands(image):
        temp = image.select('temperature_2m')
        dewpoint = image.select('dewpoint_temperature_2m')
        u_wind = image.select('u_component_of_wind_10m')
        v_wind = image.select('v_component_of_wind_10m')

        e_temp = temp.expression(
            '6.112 * exp((17.67 * temp) / (temp + 243.5))',
            {'temp': temp}
        )

        e_dew = dewpoint.expression(
            '6.112 * exp((17.67 * dewpoint) / (dewpoint + 243.5))',
            {'dewpoint': dewpoint}
        )

        rh = e_dew.divide(e_temp).multiply(100).rename('relative_humidity_2m')

        wind_speed = u_wind.pow(2).add(v_wind.pow(2)).sqrt().rename('wind_speed_10m')

        return image.addBands([rh, wind_speed])

    # Apply the relative humidity calculation
    era5_with_rh = era5_selected.map(calculate_additional_bands)

    # Function to reduce each image to a dictionary of values
    def reduce_image(image, feature):
        reduced = image.reduceRegion(
            reducer=ee.Reducer.mode(),
            geometry=feature.geometry(),
            scale=1000,
            maxPixels=1e9
        )
        return ee.Feature(None, reduced).set('date', image.date().format('YYYY-MM-dd')).copyProperties(feature, ['Fire_ID', 'defoltd', 'id'])

    # Function to process each feature in the collection
    def process_feature(feature):
        era5_reduced = era5_with_rh.map(lambda image: reduce_image(image, feature))
        return era5_reduced

    # Apply the processing function to each feature in the collection
    processed_collection = feature_collection.map(process_feature).flatten()

    # Convert the collection to a list of dictionaries
    data_list = processed_collection.getInfo()['features']

    # Extract properties from each feature
    data_dicts = [feature['properties'] for feature in data_list]

    # Create a DataFrame
    df = pd.DataFrame(data_dicts)

    df = df[['date', 'temperature_2m', 'relative_humidity_2m', 'wind_speed_10m',
             'total_precipitation', 'Fire_ID', 'defoltd', 'id']]

    def kelvin_to_celcius(x):
        return x - 273.15

    df['temperature_2m'] = df['temperature_2m'].apply(kelvin_to_celcius)

    return df